In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import *

# Definizione funzioni

In [ ]:
#Per la sezione di polarizzazione

def Malus(alfa, I0):
    return I0*np.cos(alfa)**2

#Sono da inserire di seguito le funzioni per fit dell'esperienza di polarizzazione
#Sono le funzioni al variare di theta e r 

#Per Brewster bisogna utilizzare un'iterpolazione parabolica
def parabola(x, a, b, c):
    return a*x**2 + b*x + c

#Per la sezione di interferenza dobbiamo scegliere due esperienza diverse. 
#Riporto comunque le funzioni per tutte e quattro

#1 Per la sezione di interferenza con la doppia fenditura
#massimi sono a: d*sin(theta) = n*lambda. Voglio ottenere theta
def double_slit_max(n, d, l):
    return np.arcsin(n*l/d)
    
#minimi sono a: d*sin(theta) = (n/2)*lambda. Voglio ottenere theta
def double_slit_min(n, d, l):
    return np.arcsin(n*l/(2*d))


#2 Per la sezione di interferenza con specchio di LLoyd

#voglio ottenere lambda
def lloyd(h1, h2, d1):
    return 2*(np.sqrt(h2**2 + d1**2) - np.sqrt(h1**2 + d1**2))

#3 Per la sezione di interferenza con cavità di Fabry-Perot

#voglio ottenere distanza tra i due schermi semiriflettenti
def fabry_perot(l, m):
    return m*l/2

#4 Per la sezione di interferenza con Michelson

#voglio ottenere lambda. N è il numero di frange contate nello spostamento x dello specchio
def michelson(N, x):
    return 2*x/N

# Per la sezione di diffrazione alla Bragg

#Il massimo si ha per 2*d*sin(theta) = n*lambda. Voglio ottenere d
def braggs(l, n, theta):
    return l/(2*np.sin(theta))

# Import dati

In [ ]:
dati1 = pd.read_excel("dati_pol.xlsx")

# POLARIZZAZIONE

**NOTA!!!**\
ricordiamoci dell'errore sulle x

Le misure degli angoli sono prese in gradi \
Le misure di intensità sono prese in volt \
Sensibilità goniometro = 5 gradi \
Sensibilità amperometro = 0.01 volt \
Oscillazione amperometro = 0.05 \
rrore sistematico nel goniomoetro: non è possibile fare in modo che siano sempre dritti /
Tentiamo d prendere le misure con sempre la stessa inclinazione.

Le misure degli angoli sono prese in gradi \
Le misure di intensità sono prese in volt \
Sensibilità goniometro = 5 gradi \
Sensibilità amperometro = 0.01 volt \
Oscillazione amperometro = 0.05 \
rrore sistematico nel goniomoetro: non è possibile fare in modo che siano sempre dritti /
Tentiamo d prendere le misure con sempre la stessa inclinazione.

Le misure degli angoli sono prese in gradi \
Le misure di intensità sono prese in volt \
Sensibilità goniometro = 5 gradi \
Sensibilità amperometro = 0.01 volt  \
Oscillazione amperometro = 0.05 \
Errore sistematico nel goniometro: non è possibile fare in modo che siano sempre dritti \
Tentiamo d prendere le misure con sempre la stessa inclinazione. \
Distanza tra i ricevitori (da dritti) = 50cm

# AMPIEZZA

## Rotazione

In [ ]:
dati_rotazione = pd.read_excel("dati_rotazione.xlsx")
gradi_rotaz_prec = dati_rotazione["gradi_precisi"]
amp2 = dati_rotazione["tensione_precisa"]
amp2_err = [0.05 for _ in amp2]

In [ ]:
plt.errorbar(gradi_rotaz_prec, amp2, amp2_err)
plt.show()

## Distanza

In [ ]:
dati_distanza = pd.read_excel("dati_distanza.xlsx")
dist = dati_distanza["distanza"][7:]
amp3 = dati_distanza["tensione"][7:]

In [ ]:
plt.scatter(dist, amp3, marker=".")
plt.show()

In [ ]:
# plot con le misure più fitte
# le misure oscillavano di meno

dist_prec = dati_distanza["distanza_precisa"].dropna()
amp3_prec = dati_distanza["tensione_precisa"].dropna()
amp3_prec_err = [0.02 for _ in amp3_prec]

In [ ]:
plt.errorbar(dist_prec, amp3_prec, amp3_prec_err, fmt=".r")
plt.grid()
plt.show()

proviamo a fittare con un modello combinato tra uno che va come 1/r o 1/r^2 e uno che va con un coseno smorzato

# ANGOLO DI BREWSTER

tensione senza polietilene $\approx$ 2.95 \
errore sulle x (gradi) = 2-3 \

In [ ]:
dati_brewster_trasm = pd.read_excel("dati_brewster_trasm.xlsx")
gradi_br = dati_brewster_trasm["angolo"]
amp_br_tr_oriz = dati_brewster_trasm["tensione(oriz)"]
amp_br_tr_vert = dati_brewster_trasm["tensione(vert)"]
amp_br_tr_err = [0.03 for _ in amp_br_tr_vert]

## Trasmissione

In [ ]:
plt.errorbar(gradi_br, amp_br_tr_oriz, amp_br_tr_err)
plt.grid()
plt.show()

per spiegare il picco iniziale, le onde vengono riflesse più volte, quindi leggiamo un segnale maggiore. \
per spiegare il picco finale, semplicemente parte delle onde raggiunge il ricevitore inimpedita, quindi non c'è perdita di segnale

In [ ]:
plt.errorbar(gradi_br, amp_br_tr_vert, amp_br_tr_err)
plt.grid()
plt.show()

## Riflessione

L'angolo lo abbiamo preso in base alla lastra di polietilene: ruotando la lastra, misuriamo l'angolo e spostiamo l'emettitore in modo da avere 180-2*$\theta$ (per la legge degli angoli o qualcosa) \
Ad esempio: se lo specchio è a 30°, l'emettitore sarà a 180°-60° = 120°

In [ ]:
# i dati non sono per nulla affidabili, se vogliamo fare sta parte sono da riprendere

# DIFFRAZIONE DI BRAGG

# INTERFERENZA